<center><h1>multiprocessing</h1></center>

In [2]:
import multiprocessing as mp

# 1 Process
---
```python
from multiprocessing import Process
```

## 1.1 introduction
---
1. 需要将要做的事件进行封装函数
2. 使用`multiprocessing`提供的类`Process`创建进程对象。
3. 通过对象和`Process`的初始化函数对进程进行设置以及绑定要执行的事件。
4. 启动进程，会自动的执行函数代表的时间。
5. 完成进程的回收。

> help(mp.Process)

```python
Process(group=None, target=None, name=None, args=(), kwargs={}, *, daemon=None)
```
- name: 给创建的进程定义名字，默认为Process-1...
- target: 绑定的函数
- args: 元组， target函数的位参
- kwargs: 关键字传参
- daemon: 如果为True,则主进程退出前，结束所有子进程运行。default False.
```python
p.start()
```
- 启动进程， 进程被创建，自动运行对应的函数。

```python
p.join([timeout])
```
- 阻塞等待对应子进程退出，回收子进程。


```python
p.is_alive(self) # return whether process is alive

p.run(self) # 用于自定义Process重写. myProcess.py

p.terminate(self) # 直接让p进程结束。terminate process;sends SIGTERM signal or uses TerminateProcess()

p.authkey

p.daemon 

p.exitcode

p.ident # pid

p.name # p name

p.pid # pid

p.sentinel # Return a file descriptor (Unix) or handle (Windows) suitable for waiting for process termination.
```


> **注意**

- 如果不使用`join()`则子进程可能会成为**僵尸进程**。
- 在使用`multiprocessing`创建进程中，一般父进程功能就是创建子进程等待回收，不做其他的功能。

## 1.2 附件
---
- <span style="color:red">进程线程相关的代码不要直接在notebook上执行！输出会有异常。</span>
- `myProcess.py`

# 2 进程池
---
```python
from multiprocessing import Pool
```


## 2.1 introduction
---

1. 进程池产生原因
    - 如果有大量的任务需要多进程完成，则可能需要频繁的创建和删除进程，给计算机带来额外的消耗。
    - **使用于**大量且运行时间短的进程。
2. 操作流程

```python
pool = Pool(processes) # create process pool.
event_obj = pool.apply_async(func, args, kwds) # add event to process pool.
    # params:
    # ---
    # func: function name that want to invoke.
    # args: a tuple of args
    # kwds:
    # return: 返回一个事件对象，通过`get()`可以获取事件执行函数的返回值。

pool.close() # close process pool,
pool.join() #recyle processes.
```

## 2.2 other
---
```python
pool.apply() # 同步执行func, 即一个个的执行，`p.apply_async()`:异步执行
pool.map(func, iter)
pool.map.async()
```

## 2.3 附件
---
`mypool.py`

# 3 进程间通信
---
## 3.1 introduction
---
1. 进程间由于空间独立，资源互相无法直接获取，此时在不同的进程间传递数据就需要专门的进程间通信方法。
2. 进程间的通信方法(IPC)
    - 管道，队列，共享内存，信号，信号量，套接字
    
## 3.2 管道
---
```python
from multiprocessing import Pipe
```
- 在内存中开辟一块空间，形成管道结构，管道对多个进程可见，进程可以通过对管道的读写操作进行通信。

```python
(conn1, conn2) = Pipe(duplex=True)
     # duplex:默认为双向管道，如果为`False`则为单向管道。
     # 如果为单向管道，`conn1`只能进行读操作。`conn2`只能进行写操作。
     # `conn`为`Connection`类的实例，可以使用的方法有
        # send(obj)
        # recv()
        # fileno()
        # close()
        # poll([timeout])
        # send_bytes(buffer)
        # recv_bytes()
        # recv_bytes_into(buffer)
```

- **附件** `myPipe.py`
    
## 3.3 队列
---
```python
from multiprocessing import Queue
```
- **先进先出**。在内存中开辟队列空间结构，对多个进程可见。多个进程向队列中存入消息，取出消息，完成进程间通信。

> **Queue**

```python
q = Queue([maxsize])

q.put(obj[, block[, timeout]]) # if full raise `queue.Full` exception
q.put_nowait(obj) # equal `put(obj, False)`不阻塞，如果放满则异常。
 
q.get([block[, timeout]]) # if empty raise `queue.Empty` exception
q.get_nowait()` # equal `get(False)`不阻塞，取不到值则异常。
 
q.qsize() # Return the **approximate** size of the queue.
q.empty() # Return True if the queue is empty.
q.full() # Return True if the queue is full.
q.close()
 
q.join_thread() # 添加后台线程，必须放在`close()`方法之后，如果管道内的还有数据没有get，则会一直阻塞。？？？
q.cancel_join_thread()
```
    
> **附件** `myQueue.py`

## 3.4 共享内存
---
```python
from multiprocessing import Value
from multiprocessing import Array
```
- 在内存中开辟一段空间，存储数据，对多个进程可见，每次写入共享内存的数据会覆盖之前的内容。

> **Value**

```python
obj = Value(ctype, obj) # 开辟共享内存空间
    # ctype: 字符串，要转变的C的类型code(对照ctype表)
    # obj: 共享内存初始值
obj.value # 表示共享内存中的值。对其修改或者使用即为使用共享内存中的值。
```

> **Array**

```python
obj = Array(ctype, obj) # 开辟共享内存空间
    # ctype: 要转换的类型
    # obj: 要存入共享内存的数据，如果是列表，则类型必须一直。如果是整数，则开辟整数长度的ctype类型的列表。
```
> **附件** `myValue.py`, `myArray.py`


|--|管道|  消息队列|共享内存|
|--|:--|--|--|
|开辟空间|内存|内存|内存|
|读写方式|两端读写，双向或单向|先进先出|操作覆盖内存|
|效率|一般|一般|较快|
|应用|多用于父子进程|广泛灵活 | 复杂，需要互斥机制|